In [1]:
import pandas as pd
import re
from datetime import datetime
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors


In [2]:
df_foursq= pd.read_csv("datos/api_foursquare.csv", index_col=0)
df_aemet= pd.read_csv("datos/df_aemet_final.csv")
df_municipios= pd.read_csv("datos/municipios.csv", index_col=0)

In [3]:
df_municipios.sample(2)

,acebeda-la,ajalvir,alameda-del-valle,alamo-el,alcala-de-henares,alcobendas,alcorcon,aldea-del-fresno,algete,alpedrete,...,villamanta,villamantilla,villanueva-de-la-canada,villanueva-del-pardillo,villanueva-de-perales,villar-del-olmo,villarejo-de-salvanes,villaviciosa-de-odon,villavieja-del-lozoya,zarzalejo
0,41.0869583,40.5342302,40.9177178,40.2307296,40.4818396,40.5400082,40.3492998,40.3215212,40.5961652,40.6599005,...,40.2990939,40.3421033,40.4472106,40.4949232,40.3466066,40.3364465,40.1683582,40.3573787,41.0063622,40.5483016
2,La Acebeda,Ajalvir,Alameda del Valle,El Álamo,Alcalá de Henares,Alcobendas,Alcorcón,Aldea del Fresno,Algete,Alpedrete,...,Villamanta,Villamantilla,Villanueva de la Cañada,Villanueva del Pardillo,Villanueva de Perales,Villar del Olmo,Villarejo de Salvanés,Villaviciosa de Odón,Villavieja del Lozoya,Zarzalejo


In [4]:
df_foursq["id_municipio_cambiar"]= df_foursq["municipio"]
df_foursq.drop(columns="municipio", inplace=True)

In [5]:
df_foursq.sample(2)

,category,fsq_id,closed_bucket,distance,link,name,address,latitude,longitude,id_municipio_cambiar
52,Monument,9d866cc87d374d9f58f4dc33,LikelyOpen,356.0,/v3/places/9d866cc87d374d9f58f4dc33,Convento de San Juan de la Penitencia,"Calle Santiago, N, 3, 28801 Alcalá de Henares ...",40.482736,-3.368547,alcala-de-henares
366,Park,5326e731498e591bbc11b95a,VeryLikelyOpen,1892.0,/v3/places/5326e731498e591bbc11b95a,Puerta de Madrid de los Jardines del Buen Retiro,Madrid Comunidad de Madrid,40.420794,-3.681865,madrid


In [6]:
df_aemet.sample(2)

,fecha,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento,localizacion_id
4187,2024-08-29 13:00:00,Nubes altas,24,24,29.0,0.0,0,56,Riesgo,0,75,Riesgo,SE,11,navarredonda-y-san-mames-id28097
2911,2024-08-29 13:00:00,Nubes altas,24,24,21.0,0.0,0,66,Riesgo,0,70,Riesgo,E,9,guadalix-de-la-sierra-id28067


Separamos localización id 

In [7]:
df_aemet["localizacion_id"]

0       acebeda-la-id28001
1       acebeda-la-id28001
2       acebeda-la-id28001
3       acebeda-la-id28001
4       acebeda-la-id28001
               ...        
7880     zarzalejo-id28183
7881     zarzalejo-id28183
7882     zarzalejo-id28183
7883     zarzalejo-id28183
7884     zarzalejo-id28183
Name: localizacion_id, Length: 7885, dtype: object

In [8]:
df_aemet["id_municipio"]=pd.DataFrame(df_aemet["localizacion_id"].str.split("\.*-id\.*").str[1])
df_aemet["id_localizacion"]= pd.DataFrame(df_aemet["localizacion_id"].str.split("\.*-id\.*").str[0])
df_aemet.drop(columns="localizacion_id", inplace=True)



In [9]:
df_aemet.head(2)

,fecha,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento,id_municipio,id_localizacion
0,2024-08-29 12:00:00,Nubes altas,21,21,25.0,0.0,0,69,95,0,75,Riesgo,S,7,28001,acebeda-la
1,2024-08-29 13:00:00,Nubes altas,24,24,29.0,0.0,0,57,Riesgo,0,75,Riesgo,SE,12,28001,acebeda-la


In [10]:
def crear_df_id (df, col):
    df_nuevo= pd.DataFrame(df[col])
    df_nuevo[f"id_{col}"]= pd.RangeIndex(start=1,stop = len(df_nuevo)+1,step=1)
    return df_nuevo

Traspongo municipios para que se convierta el municipio en su id

creamos dicc entre id_mun y id_loc

In [11]:
lista_mun=[]
for nombre in df_aemet["id_localizacion"].unique():
    lista_mun.append(nombre)
# lista_mun  

Nombro las columnas

Meter id_municipio en tods las tablas

In [15]:
df_aemet.head(4)

,fecha,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento,id_municipio,id_localizacion
0,2024-08-29 12:00:00,Nubes altas,21,21,25.0,0.0,0,69,95,0,75,Riesgo,S,7,28001,acebeda-la
1,2024-08-29 13:00:00,Nubes altas,24,24,29.0,0.0,0,57,Riesgo,0,75,Riesgo,SE,12,28001,acebeda-la
2,2024-08-29 14:00:00,Muy nuboso,23,23,33.0,0.0,0,59,100,0,80,Riesgo,S,14,28001,acebeda-la
3,2024-08-29 15:00:00,Nubes altas,23,23,35.0,0.0,0,55,Riesgo,0,80,Riesgo,S,17,28001,acebeda-la


Monto las tablas que no tienen foreign keys

In [17]:
def crear_df_id (df, col):
    df_nuevo= pd.DataFrame(df[col])
    df_nuevo[f"id_{col}"]= pd.RangeIndex(start=1,stop = len(df_nuevo)+1,step=1)
    return df_nuevo

In [18]:
lista_cielo=[]
for nombre in df_aemet["cielo"].unique():
    lista_cielo.append(nombre)
lista_cielo   

['Nubes altas',
 'Muy nuboso',
 'Cubierto con tormenta y lluvia escasa',
 'Nuboso',
 'Cielo despejado',
 'Poco nuboso',
 'Intervalos nubosos con lluvia escasa',
 'Cubierto',
 'Cubierto con lluvia escasa',
 'Cubierto con lluvia',
 'Muy nuboso con tormenta y lluvia escasa',
 'Muy nuboso con lluvia',
 'Intervalos nubosos',
 'Muy nuboso con lluvia escasa',
 'Nuboso con tormenta y lluvia escasa',
 'Cubierto con tormenta',
 'Nuboso con lluvia escasa',
 'Intervalos nubosos con lluvia',
 'Bruma',
 'Niebla',
 'Nuboso con lluvia',
 'Muy nuboso con tormenta',
 'Nuboso con tormenta']

In [19]:
df_cielo1= pd.DataFrame(lista_cielo)
df_cielo1= df_cielo1.rename(columns={0 : "cielo"})
df_cielo= crear_df_id(df_cielo1,"cielo")
df_cielo.head()

,cielo,id_cielo
0,Nubes altas,1
1,Muy nuboso,2
2,Cubierto con tormenta y lluvia escasa,3
3,Nuboso,4
4,Cielo despejado,5


In [20]:
lista_avisos=[]
for nombre in df_aemet["avisos"].unique():
    lista_avisos.append(nombre)
lista_avisos   

['Riesgo', 'Sin riesgo']

In [21]:
df_avisos1= pd.DataFrame(lista_avisos)
df_avisos1= df_avisos1.rename(columns={0 : "avisos"})
df_avisos= crear_df_id(df_avisos1,"avisos")
df_avisos.head()

,avisos,id_avisos
0,Riesgo,1
1,Sin riesgo,2


In [22]:
lista_categorias=[]
for nombre in df_foursq["category"].unique():
    lista_categorias.append(nombre)
lista_categorias

['Park',
 'Monument',
 'TV Station',
 'Castle',
 'Film Studio',
 'Palace',
 'Fountain',
 'Bridge',
 'Beach',
 'Print, TV, Radio and Outdoor Advertising Service']

In [23]:
df_categorias1= pd.DataFrame(lista_categorias)
df_categorias1= df_categorias1.rename(columns={0 : "categorias"})
df_categorias= crear_df_id(df_categorias1,"categorias")
df_categorias.head()

,categorias,id_categorias
0,Park,1
1,Monument,2
2,TV Station,3
3,Castle,4
4,Film Studio,5


In [24]:
lista_estados=[]
for nombre in df_foursq["closed_bucket"].unique():
    lista_estados.append(nombre)
lista_estados

['LikelyOpen', 'VeryLikelyOpen', 'Unsure']

In [25]:
df_estados1= pd.DataFrame(lista_estados)
df_estados1= df_estados1.rename(columns={0 : "estados"})
df_estados= crear_df_id(df_estados1,"estados")
df_estados.head()

,estados,id_estados
0,LikelyOpen,1
1,VeryLikelyOpen,2
2,Unsure,3


Pasamos a las siguientes tablas con menos ids 2ºtiempo

In [31]:
df= pd.DataFrame()
df["fecha"]= df_aemet["fecha"]
df["temperatura_°c"]= df_aemet["temp._(°c)"]
df["precipitacion_mm"]= df_aemet["precipitación_(mm)"]
df["humedad_%"]= df_aemet["humedad_relativa_(%)"]
df["avisos"]= df_aemet["avisos"]
df["cielo"]= df_aemet["cielo"]
df["id_localizacion"]= df_aemet["id_localizacion"]
df["id_municipio"]= df_aemet["id_municipio"]
df_tiempo= df
df_tiempo.head(2)

,fecha,temperatura_°c,precipitacion_mm,humedad_%,avisos,cielo,id_localizacion,id_municipio
0,2024-08-29 12:00:00,21,0.0,69,Riesgo,Nubes altas,acebeda-la,28001
1,2024-08-29 13:00:00,24,0.0,57,Riesgo,Nubes altas,acebeda-la,28001


In [32]:
diccionario_mun= {}
datos = df_aemet.groupby("id_localizacion")["id_municipio"].first()
datos_cielo = list(datos.index)
lista_estados = list(datos.values)
for i in range(0, len(datos_cielo)):
    diccionario_mun[datos_cielo[i]] = lista_estados[i]

df_tiempo["id_municipio"] = df_tiempo["id_localizacion"].map(diccionario_mun)

In [33]:
diccionario= {}
datos = df_cielo.groupby("cielo")["id_cielo"].first()
datos_cielo = list(datos.index)
lista_estados = list(datos.values)
for i in range(0, len(datos_cielo)):
    diccionario[datos_cielo[i]] = lista_estados[i]

df_tiempo["id_cielo"] = df_tiempo["cielo"].map(diccionario)

In [34]:

diccionario= {}
datos = df_avisos.groupby("avisos")["id_avisos"].first()
datos_cielo = list(datos.index)
lista_estados = list(datos.values)
for i in range(0, len(datos_cielo)):
    diccionario[datos_cielo[i]] = lista_estados[i]

df_tiempo["id_avisos"] = df_tiempo["avisos"].map(diccionario)

df_tiempo.drop(columns="cielo", inplace=True)
df_tiempo.drop(columns="avisos", inplace=True)  
df_tiempo.drop(columns="id_localizacion", inplace=True)  

In [35]:
df_tiempo.index = pd.RangeIndex(start=1,stop = len(df_tiempo)+1,step=1)
df_tiempo["id_tiempo"]=df_tiempo.index
df_tiempo.head()


,fecha,temperatura_°c,precipitacion_mm,humedad_%,id_municipio,id_cielo,id_avisos,id_tiempo
1,2024-08-29 12:00:00,21,0.0,69,28001,1,1,1
2,2024-08-29 13:00:00,24,0.0,57,28001,1,1,2
3,2024-08-29 14:00:00,23,0.0,59,28001,2,1,3
4,2024-08-29 15:00:00,23,0.0,55,28001,1,1,4
5,2024-08-29 16:00:00,21,0.0,65,28001,1,1,5


tabla mun

In [36]:
df_municipioss= df_municipios.T
df_municipioss=df_municipioss.reset_index()
df_municipioss.head()


,index,0,1,2
0,acebeda-la,41.0869583,-3.624399,La Acebeda
1,ajalvir,40.5342302,-3.4807818,Ajalvir
2,alameda-del-valle,40.9177178,-3.8438216,Alameda del Valle
3,alamo-el,40.2307296,-3.9905893,El Álamo
4,alcala-de-henares,40.4818396,-3.3644973,Alcalá de Henares


In [37]:
lista= ["id_municipio_cambiar","latitude","longitude","nombre_municipio"]
df_municipioss.columns= lista
df_municipioss.head(2)


,id_municipio_cambiar,latitude,longitude,nombre_municipio
0,acebeda-la,41.0869583,-3.624399,La Acebeda
1,ajalvir,40.5342302,-3.4807818,Ajalvir


In [38]:
df= pd.DataFrame()
df["nombre_municipio"]= df_municipioss["nombre_municipio"]
df["latitud"]= df_municipioss["latitude"]
df["longitud"]= df_municipioss["longitude"]
df["id_union"]= df_municipioss["id_municipio_cambiar"]
df_municipios1= df
df_municipios1.head(2)


,nombre_municipio,latitud,longitud,id_union
0,La Acebeda,41.0869583,-3.624399,acebeda-la
1,Ajalvir,40.5342302,-3.4807818,ajalvir


In [42]:



df_municipios1["id_municipio"] = df_municipios1["id_union"].map(diccionario_mun)
df_municipios1.drop(columns="id_union", inplace=True) 


In [44]:
df_municipios1.head(2)

,nombre_municipio,latitud,longitud,id_municipio
0,La Acebeda,41.0869583,-3.624399,28001
1,Ajalvir,40.5342302,-3.4807818,28002


Hacemos la última tabla (lugar)

In [46]:
df_foursq.head()

,category,fsq_id,closed_bucket,distance,link,name,address,latitude,longitude,id_municipio_cambiar
0,Park,4d3c6c5b84d46ea87dddfd5c,LikelyOpen,563.0,/v3/places/4d3c6c5b84d46ea87dddfd5c,Diverjungla,"Calle Segovia, 20 (Pol. Ind. Donada), 28864 Aj...",40.529921,-3.484341,ajalvir
1,Park,4f54f7bfe4b036244d02685b,VeryLikelyOpen,1724.0,/v3/places/4f54f7bfe4b036244d02685b,Parque Infantil,NaN,40.543867,-3.464773,ajalvir
2,Park,7d4ef99ca2dd4789f032a90c,LikelyOpen,333.0,/v3/places/7d4ef99ca2dd4789f032a90c,Chiquitin,Ajalvir Madrid,40.531662,-3.482835,ajalvir
3,Park,53de1144498e8fd8d8cb3537,LikelyOpen,118.0,/v3/places/53de1144498e8fd8d8cb3537,Isla Perejil,28607 El Alamo Comunidad de Madrid,40.230993,-3.991942,alamo-el
4,Park,51a8f451498ecb8334de37df,LikelyOpen,582.0,/v3/places/51a8f451498ecb8334de37df,Parque de Madera,NaN,40.234636,-3.995166,alamo-el


In [47]:
df= pd.DataFrame()
df["nombre"]= df_foursq["name"]
df["distancia"]= df_foursq["distance"]
df["categoria"]= df_foursq["category"]
df["estado"]= df_foursq["closed_bucket"]
df["id_localizacion"]= df_foursq["id_municipio_cambiar"]
df_lugar= df
df_lugar.head(2)

,nombre,distancia,categoria,estado,id_localizacion
0,Diverjungla,563.0,Park,LikelyOpen,ajalvir
1,Parque Infantil,1724.0,Park,VeryLikelyOpen,ajalvir


In [48]:


df_lugar["id_municipio"] = df_lugar["id_localizacion"].map(diccionario_mun)

In [49]:

diccionario= {}
datos = df_categorias.groupby("categorias")["id_categorias"].first()
datos_cielo = list(datos.index)
lista_estados = list(datos.values)
for i in range(0, len(datos_cielo)):
    diccionario[datos_cielo[i]] = lista_estados[i]

df_lugar["id_categoria"] = df_lugar["categoria"].map(diccionario)

In [50]:

diccionario= {}
datos = df_estados.groupby("estados")["id_estados"].first()
datos_cielo = list(datos.index)
lista_estados = list(datos.values)
for i in range(0, len(datos_cielo)):
    diccionario[datos_cielo[i]] = lista_estados[i]

df_lugar["id_estados"] = df_lugar["estado"].map(diccionario)

df_lugar.drop(columns="categoria", inplace=True)
df_lugar.drop(columns="estado", inplace=True) 
df_lugar.drop(columns="id_localizacion", inplace=True) 

In [51]:
df_lugar.index = pd.RangeIndex(start=1,stop = len(df_lugar)+1,step=1)
df_lugar["id_lugar"]=df_lugar.index
df_lugar.head()

,nombre,distancia,id_municipio,id_categoria,id_estados,id_lugar
1,Diverjungla,563.0,28002,1,1,1
2,Parque Infantil,1724.0,28002,1,2,2
3,Chiquitin,333.0,28002,1,1,3
4,Isla Perejil,118.0,28004,1,1,4
5,Parque de Madera,582.0,28004,1,1,5


Comprobamos nulos y tipos de cada DF

In [53]:
df_tiempo["id_municipio"]=df_tiempo["id_municipio"].astype(int)

In [54]:
df_tiempo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7885 entries, 1 to 7885
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   fecha             7885 non-null   object 
 1   temperatura_°c    7885 non-null   int64  
 2   precipitacion_mm  7885 non-null   float64
 3   humedad_%         7885 non-null   int64  
 4   id_municipio      7885 non-null   int64  
 5   id_cielo          7885 non-null   int64  
 6   id_avisos         7885 non-null   int64  
 7   id_tiempo         7885 non-null   int64  
dtypes: float64(1), int64(6), object(1)
memory usage: 492.9+ KB


In [58]:
df_municipios1["latitud"]= df_municipios1["latitud"].astype(float,)
df_municipios1["longitud"]= df_municipios1["longitud"].astype(float)
df_municipios1["id_municipio"]=df_municipios1["id_municipio"].astype(int)

In [59]:
df_municipios1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   nombre_municipio  176 non-null    object 
 1   latitud           176 non-null    float64
 2   longitud          176 non-null    float64
 3   id_municipio      176 non-null    int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 5.6+ KB


In [61]:
df_lugar["id_municipio"]=df_lugar["id_municipio"].astype(int)

In [62]:
df_lugar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 907 entries, 1 to 907
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   nombre        907 non-null    object 
 1   distancia     907 non-null    float64
 2   id_municipio  907 non-null    int64  
 3   id_categoria  907 non-null    int64  
 4   id_estados    907 non-null    int64  
 5   id_lugar      907 non-null    int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 42.6+ KB


**PASEMOS A CREAR LAS TABLAS**

In [79]:
try:
    conexion= psycopg2.connect(
        database= "Lab.14",
        user= "postgres",
        password= "admin",
        host= "localhost",
        port= "5432"
    )
except OperationalError as e:
    if e.pgcode == errorcodes.INVALID_PASSWORD:
        print("La contraseña es erronea")
    elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
        print("Error de conexion")
    else:
        print(f"Ocurrió el error {e}")

TABLA CIELO

In [64]:
cursor= conexion.cursor()
query_creacion= """
CREATE TABLE IF NOT EXISTS cielo(
    cielo VARCHAR(100),
    id_cielo INT PRIMARY KEY
    
);
"""
cursor.execute(query_creacion)   
conexion.commit()

In [65]:
lista_de_tuplas=[]
for fila in df_cielo.values:
    lista_de_tuplas.append(tuple(fila))


In [66]:
     
query_insercion= "INSERT INTO cielo (cielo, id_cielo) VALUES (%s, %s);"      #%s es como el format de los strings (la f)

cursor.executemany(query_insercion, lista_de_tuplas)                 #Executemany es para poder insertar varios valores
conexion.commit()

TABLA AVISOS

In [67]:
cursor= conexion.cursor()
query_creacion= """
CREATE TABLE IF NOT EXISTS avisos(
    avisos VARCHAR(100),
    id_avisos INT PRIMARY KEY
    
);
"""
cursor.execute(query_creacion)   
conexion.commit()

In [68]:
lista_de_tuplas=[]
for fila in df_avisos.values:
    lista_de_tuplas.append(tuple(fila))


In [69]:
     
query_insercion= "INSERT INTO avisos (avisos, id_avisos) VALUES (%s, %s);"      #%s es como el format de los strings (la f)

cursor.executemany(query_insercion, lista_de_tuplas)                 #Executemany es para poder insertar varios valores
conexion.commit()

TABLA CATEGORIA

In [70]:
cursor= conexion.cursor()
query_creacion= """
CREATE TABLE IF NOT EXISTS categorias(
    categorias VARCHAR(100),
    id_categorias INT PRIMARY KEY
    
);
"""
cursor.execute(query_creacion)   
conexion.commit()

In [71]:
lista_de_tuplas=[]
for fila in df_categorias.values:
    lista_de_tuplas.append(tuple(fila))


In [72]:
     
query_insercion= "INSERT INTO categorias (categorias, id_categorias) VALUES (%s, %s);"      #%s es como el format de los strings (la f)

cursor.executemany(query_insercion, lista_de_tuplas)                 #Executemany es para poder insertar varios valores
conexion.commit()

TABLA ESTADO

In [73]:
cursor= conexion.cursor()
query_creacion= """
CREATE TABLE IF NOT EXISTS estados(
    estados VARCHAR(100),
    id_estados INT PRIMARY KEY
    
);
"""
cursor.execute(query_creacion)   
conexion.commit()

In [74]:
lista_de_tuplas=[]
for fila in df_estados.values:
    lista_de_tuplas.append(tuple(fila))


In [75]:
     
query_insercion= "INSERT INTO estados (estados, id_estados) VALUES (%s, %s);"      #%s es como el format de los strings (la f)

cursor.executemany(query_insercion, lista_de_tuplas)                 #Executemany es para poder insertar varios valores
conexion.commit()

TABLA MUNICIPIO

In [80]:
cursor= conexion.cursor()
query_creacion= """
CREATE TABLE IF NOT EXISTS municipios(
    nombre_municipio VARCHAR(100),
    latitud NUMERIC(8,6),
    longitud NUMERIC(8,6),
    id_municipio INT PRIMARY KEY
    
);
"""
cursor.execute(query_creacion)   
conexion.commit()

In [81]:
lista_de_tuplas=[]
for fila in df_municipios1.values:
    lista_de_tuplas.append(tuple(fila))


In [82]:
     
query_insercion= "INSERT INTO municipios (nombre_municipio, latitud, longitud, id_municipio) VALUES (%s, %s, %s, %s);"      #%s es como el format de los strings (la f)

cursor.executemany(query_insercion, lista_de_tuplas)                 #Executemany es para poder insertar varios valores
conexion.commit()

**AHORA TOCAN LAS QUE CUENTAN CON FOREIGN KEYS**

In [ ]:
cursor= conexion.cursor()
query_creacion= """
CREATE TABLE IF NOT EXISTS tiempo(
    fecha TIMESTAMP,
    temperatura_°c INT,
    precipitacion_mm NUMERIC(8,6),
    humedad INT,
    id_avisos INT,
    id_cielo INT,
    id_municipios INT,
    id_tiempo INT PRIMARY KEY,
FOREIGN KEY (id_municipios) REFERENCES municipios(id_municipios) ON DELETE RESTRICT ON UPDATE CASCADE,
FOREIGN KEY (id_cielo) REFERENCES cielo(id_cielo) ON DELETE RESTRICT ON UPDATE CASCADE,
FOREIGN KEY (id_avisos) REFERENCES avisos(id_avisos) ON DELETE RESTRICT ON UPDATE CASCADE
);
"""
cursor.execute(query_creacion)   
conexion.commit()

In [ ]:
lista_de_tuplas=[]
for fila in df_tiempo.values:
    lista_de_tuplas.append(tuple(fila))


In [ ]:
     
query_insercion= "INSERT INTO tiempo (fecha, temperatura_°c, precipitacion_mm, humedad, id_avisos, id_cielo, id_municipios) VALUES (%s, %s, %s, %s, %s, %s, %s);"      #%s es como el format de los strings (la f)

cursor.executemany(query_insercion, lista_de_tuplas)                 #Executemany es para poder insertar varios valores
conexion.commit()

TypeError: not all arguments converted during string formatting

TABLA LUGAR

In [ ]:
df_lugar['id_municipio'].fillna(-1, inplace=True)

In [ ]:
cursor= conexion.cursor()
query_creacion= """
CREATE TABLE IF NOT EXISTS lugar(
    nombre VARCHAR(100),
    distancia DOUBLE PRECISION,
    id_municipio INT,
    id_categoria INT,
    id_estados INT,
    id_lugar INT PRIMARY KEY,
FOREIGN KEY (id_municipio) REFERENCES municipios(id_municipios) ON DELETE RESTRICT ON UPDATE CASCADE,
FOREIGN KEY (id_categoria) REFERENCES categorias(id_categorias) ON DELETE RESTRICT ON UPDATE CASCADE,
FOREIGN KEY (id_estados) REFERENCES estados(id_estados) ON DELETE RESTRICT ON UPDATE CASCADE
);
"""
cursor.execute(query_creacion)   
conexion.commit()

In [ ]:
lista_de_tuplas=[]
for fila in df_lugar.values:
    lista_de_tuplas.append(tuple(fila))


In [ ]:
     
query_insercion= "INSERT INTO lugar (nombre, distancia, id_municipio, id_categoria, id_estados, id_lugar) VALUES (%s, %s, %s, %s, %s, %s);"      #%s es como el format de los strings (la f)

cursor.executemany(query_insercion, lista_de_tuplas)                 #Executemany es para poder insertar varios valores
conexion.commit()

ForeignKeyViolation: inserción o actualización en la tabla «lugar» viola la llave foránea «lugar_id_municipio_fkey»
DETAIL:  La llave (id_municipio)=(-1) no está presente en la tabla «municipios».


**RAZONES DE ERROR:**  
  
- El id_municipio al ser creado a mano, si se introduce en tabla tiempo y lugar, genera nulos porque son 176 ids y 900 y pico filas (Lo que habría que hacer es coger el id_localizacion que tiene la misma longitud al venir de aemet)(Como tal la tabla municipios sale bien pero sería conveniente cambiar el id a id_localizacion)  
  
- En la tabla Tiempo creo que fecha me está causando el error de que no se hayan convertido todos los argumentos durante el string formatting (+ el id_municipio que genera nulos como ya he dicho)  
  
- En la tabla LUGAR mi problema principal es por culpa de los nulos de id_municipio hasta donde yo sé  